In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%writefile worker_v6.py
#
# worker_v6.py (Certified v6.4 - Final JAX Blocker Fix)
#
# This script is the "Certified V6 Worker" for the ASTE.
# It contains all code (Engine, Scorer, Bridge, HPC Pattern, and Worker Logic)
# in a single, high-performance script.
#
# --- CELL 1: IMPORTS (Corrected) ---
import jax
import jax.numpy as jnp
from jax import lax, jit
import numpy as np
import h5py
import os
import time
import functools
import json # Used to load parameters from the hunter
import traceback
from typing import NamedTuple, Callable, Dict, Tuple, Any, List
from tqdm.auto import tqdm
from functools import partial
import sys # For command-line arguments
import hashlib
import csv

print(f"JAX backend: {jax.default_backend()}")


# --- CELL 2: JAX PYTREE DEFINITIONS (STATE & PARAMS) ---
# (Source: Technical Implementation Report)

class S_NCGL_State(NamedTuple):
    """Holds the dynamic state (the complex psi field)."""
    psi: jax.Array

class S_NCGL_Params(NamedTuple):
    """Holds all static physics and simulation parameters."""
    N_GRID: int
    T_TOTAL: float
    DT: float
    alpha: float
    beta: float
    gamma: float
    KAPPA: float
    nu: float
    sigma_k: float
    l_domain: float
    num_rays: int
    k_bin_width: float
    k_max_plot: float

class SpecOps(NamedTuple):
    """Holds all pre-computed spectral arrays."""
    kx: jax.Array
    ky: jax.Array
    k_sq: jax.Array
    gaussian_kernel_k: jax.Array
    dealias_mask: jax.Array
    prime_targets_k: jax.Array
    k_bins: jax.Array
    ray_angles: jax.Array
    k_max: float
    xx: jax.Array
    yy: jax.Array
    k_values_1d: jax.Array
    sort_indices_1d: jax.Array

class S_Coupling_Params(NamedTuple):
    """Holds all coupling parameters (e.g., for the 'bridge')."""
    OMEGA_PARAM_A: float


# --- CELL 3: HDF5 LOGGER UTILITY ---
# (Source: Technical Implementation Report)

class HDF5Logger:
    """Logs simulation metrics to an HDF5 file in chunks."""
    def __init__(self, filename, n_steps, n_grid, metrics_keys, buffer_size=100):
        self.filename = filename
        self.n_steps = n_steps
        self.metrics_keys = metrics_keys
        self.buffer_size = buffer_size
        self.buffer = {key: [] for key in self.metrics_keys}
        self.buffer['omega_sq_history'] = []
        self.write_index = 0

        with h5py.File(self.filename, 'w') as f:
            for key in self.metrics_keys:
                f.create_dataset(key, (n_steps,), maxshape=(n_steps,), dtype='f4')
            f.create_dataset('omega_sq_history', shape=(n_steps, n_grid, n_grid), dtype='f4')
            f.create_dataset('final_psi', shape=(n_grid, n_grid), dtype='c8')

    def log_timestep(self, metrics: dict):
        for key in self.metrics_keys:
            if key in metrics:
                self.buffer[key].append(metrics[key])

        if 'omega_sq_history' in metrics:
            self.buffer['omega_sq_history'].append(metrics['omega_sq_history'])

        if self.metrics_keys and len(self.buffer[self.metrics_keys[0]]) >= self.buffer_size:
            self.flush()

    def flush(self):
        if not self.metrics_keys or not self.buffer[self.metrics_keys[0]]:
            return

        buffer_len = len(self.buffer[self.metrics_keys[0]])
        start = self.write_index
        end = start + buffer_len

        with h5py.File(self.filename, 'a') as f:
            for key in self.metrics_keys:
                f[key][start:end] = np.array(self.buffer[key])
            f['omega_sq_history'][start:end] = np.array(self.buffer['omega_sq_history'])

        self.buffer = {key: [] for key in self.metrics_keys}
        self.buffer['omega_sq_history'] = []
        self.write_index = end

    def save_final_state(self, final_psi):
        with h5py.File(self.filename, 'a') as f:
            f['final_psi'][:] = np.array(final_psi)

    def close(self):
        self.flush()
        print(f"HDF5Logger closed. Data saved to {self.filename}")


# --- CELL 4: CERTIFIED V6 ANALYSIS & GEOMETRY FUNCTIONS (Corrected) ---
# (Source: Technical Implementation Report)

@jit
def jnp_construct_conformal_metric(
    rho: jnp.ndarray, coupling_alpha: float, epsilon: float = 1e-9
) -> jnp.ndarray:
    """Computes the conformal factor Omega using the ECM model."""
    alpha = jnp.maximum(coupling_alpha, epsilon)
    Omega = jnp.exp(alpha * rho) # Corrected "Geometric Fix"
    return Omega

# --- FIX V6.4: Pytrees (params, spec) are dynamic, not static ---
@jit
def compute_directional_spectrum(
    psi: jax.Array, params: S_NCGL_Params, spec: SpecOps
) -> Tuple[jax.Array, jax.Array]:
    """ Implements the "multi-ray directional sampling protocol". """
    n_grid = params.N_GRID
    k_values_1d = spec.k_values_1d
    sort_indices = spec.sort_indices_1d
    power_spectrum_agg = jnp.zeros_like(spec.k_bins)

    def body_fun(i, power_spectrum_agg):
        angle = spec.ray_angles[i]
        slice_1d = psi[n_grid // 2, :] # Simplified slice
        slice_fft = jnp.fft.fft(slice_1d)
        power_spectrum_1d = jnp.abs(slice_fft)**2
        k_values_sorted = k_values_1d[sort_indices]
        power_spectrum_sorted = power_spectrum_1d[sort_indices]
        binned_power, _ = jnp.histogram(
            k_values_sorted,
            bins=jnp.append(spec.k_bins, params.k_max_plot),
            weights=power_spectrum_sorted
        )
        return power_spectrum_agg + binned_power

    power_spectrum_total = lax.fori_loop(0, params.num_rays, body_fun, power_spectrum_agg)
    power_spectrum_norm = power_spectrum_total / (jnp.sum(power_spectrum_total) + 1e-9)
    return spec.k_bins, power_spectrum_norm

# --- FIX V6.4: Pytree (spec) is dynamic, not static ---
@jit
def compute_log_prime_sse(
    k_values: jax.Array, power_spectrum: jax.Array, spec: SpecOps
) -> jax.Array:
    """ Computes the SSE against the ln(p) targets. """
    targets_k = spec.prime_targets_k
    total_power = jnp.sum(power_spectrum)

    def find_closest_idx(target_k):
        return jnp.argmin(jnp.abs(k_values - target_k))

    target_indices = jax.vmap(find_closest_idx)(targets_k)
    target_spectrum_sparse = jnp.zeros_like(k_values).at[target_indices].set(1.0)
    target_spectrum_norm = target_spectrum_sparse / jnp.sum(target_spectrum_sparse)
    diff = power_spectrum - target_spectrum_norm
    sse = jnp.sum(diff * diff)
    # Return 1.0 (failure code) if no power is found
    return jnp.where(
        total_power > 1e-9,
        jnp.nan_to_num(sse, nan=1.0, posinf=1.0, neginf=1.0),
        1.0
    )

@jit
def jnp_calculate_entropy(rho: jax.Array) -> jax.Array:
    rho_norm = rho / jnp.sum(rho)
    rho_safe = jnp.maximum(rho_norm, 1e-9)
    return -jnp.sum(rho_safe * jnp.log(rho_safe))

@jit
def jnp_calculate_quantule_census(rho: jax.Array) -> jax.Array:
    rho_mean = jnp.mean(rho)
    rho_std = jnp.std(rho)
    threshold = rho_mean + 3.0 * rho_std
    return jnp.sum(rho > threshold).astype(jnp.float32)

# --- CORRECT: 'n' MUST be static for fftfreq ---
@partial(jit, static_argnames=('n',))
def kgrid_2pi(n: int, L: float = 1.0):
    """Creates JAX arrays for k-space grids and dealiasing mask."""
    k = 2.0 * jnp.pi * jnp.fft.fftfreq(n, d=L/n)
    kx, ky = jnp.meshgrid(k, k, indexing='ij')
    k_sq = kx**2 + ky**2
    k_mag = jnp.sqrt(k_sq)
    k_max_sim = jnp.max(k_mag)
    k_ny = jnp.max(jnp.abs(kx))
    k_cut = (2.0/3.0) * k_ny
    dealias_mask = ((jnp.abs(kx) <= k_cut) & (jnp.abs(ky) <= k_cut)).astype(jnp.float32)
    return kx, ky, k_sq, k_mag, k_max_sim, dealias_mask

@jit
def make_gaussian_kernel_k(k_sq, sigma_k):
    """Pre-computes the non-local Gaussian kernel in k-space."""
    return jnp.exp(-k_sq / (2.0 * (sigma_k**2)))

print("SUCCESS: V6 Analysis & Geometry functions defined.")


# --- CELL 5: CERTIFIED V6 PHYSICS ENGINE FUNCTIONS ---
# (Source: Technical Implementation Report)

# --- FIX V6.4: Pytree (spec) is dynamic, not static ---
@jit
def spectral_gradient_complex(field: jax.Array, spec: SpecOps) -> Tuple[jax.Array, jax.Array]:
    field_fft = jnp.fft.fft2(field)
    grad_x_fft = (1j * spec.kx * field_fft) * spec.dealias_mask
    grad_y_fft = (1j * spec.ky * field_fft) * spec.dealias_mask
    return jnp.fft.ifft2(grad_x_fft), jnp.fft.ifft2(grad_y_fft)

# --- FIX V6.4: Pytree (spec) is dynamic, not static ---
@jit
def spectral_laplacian_complex(field: jax.Array, spec: SpecOps) -> jax.Array:
    field_fft = jnp.fft.fft2(field)
    field_fft = field_fft * spec.dealias_mask
    return jnp.fft.ifft2((-spec.k_sq) * field_fft)

# --- FIX V6.4: Pytree (spec) is dynamic, not static ---
@jit
def compute_covariant_laplacian_complex(
    psi: jax.Array, Omega: jax.Array, spec: SpecOps
) -> jax.Array:
    """Computes the curved-space spatial Laplacian (Laplace-Beltrami operator)."""
    epsilon = 1e-9
    Omega_safe = jnp.maximum(Omega, epsilon)
    Omega_sq_safe = jnp.square(Omega_safe)
    g_inv_sq = 1.0 / Omega_sq_safe
    flat_laplacian_psi = spectral_laplacian_complex(psi, spec)
    curvature_modified_accel = g_inv_sq * flat_laplacian_psi
    g_inv_cubed = g_inv_sq / Omega_safe
    grad_psi_x, grad_psi_y = spectral_gradient_complex(psi, spec)
    grad_Omega_x_complex, grad_Omega_y_complex = spectral_gradient_complex(Omega, spec)
    grad_Omega_x = grad_Omega_x_complex.real
    grad_Omega_y = grad_Omega_y_complex.real
    dot_product = (grad_Omega_x * grad_psi_x) + (grad_Omega_y * grad_psi_y)
    geometric_damping = g_inv_cubed * dot_product
    spatial_laplacian_g = curvature_modified_accel + geometric_damping
    return spatial_laplacian_g

# --- FIX V6.4: Pytrees (params, coupling_params, spec) are dynamic ---
@jit
def jnp_get_derivatives(
    state: S_NCGL_State, params: S_NCGL_Params,
    coupling_params: S_Coupling_Params,
    spec: SpecOps
) -> S_NCGL_State:
    """Core EOM for the S-NCGL equation, with Geometric Feedback."""
    psi = state.psi
    rho = jnp.abs(psi)**2

    # S-NCGL Physics Terms
    rho_fft = jnp.fft.fft2(rho)
    non_local_term_k_fft = spec.gaussian_kernel_k * rho_fft
    non_local_term_k = jnp.fft.ifft2(non_local_term_k_fft * spec.dealias_mask).real
    non_local_coupling = -params.nu * non_local_term_k * psi
    local_cubic_term = -params.beta * rho * psi
    source_term = params.gamma * psi
    damping_term = -params.alpha * psi

    # Geometric Feedback
    Omega = jnp_construct_conformal_metric(rho, coupling_params.OMEGA_PARAM_A)
    spatial_laplacian_g = compute_covariant_laplacian_complex(psi, Omega, spec)
    covariant_laplacian_term = params.KAPPA * spatial_laplacian_g

    # S-NCGL EOM
    d_psi_dt = (
        damping_term + source_term + local_cubic_term +
        non_local_coupling + covariant_laplacian_term
    )
    return S_NCGL_State(psi=d_psi_dt)

# --- FIX V6.4: 'deriv_func' is the *only* static arg. Others are Pytrees. ---
@partial(jit, static_argnames=('deriv_func',))
def rk4_step(
    state: S_NCGL_State, dt: float, deriv_func: Callable,
    params: S_NCGL_Params,
    coupling_params: S_Coupling_Params,
    spec: SpecOps
) -> S_NCGL_State:
    """Performs a single 4th-Order Runge-Kutta step."""
    k1 = deriv_func(state, params, coupling_params, spec)
    k2_state = jax.tree_util.tree_map(lambda y, dy: y + dy * dt / 2.0, state, k1)
    k2 = deriv_func(k2_state, params, coupling_params, spec)
    k3_state = jax.tree_util.tree_map(lambda y, dy: y + dy * dt / 2.0, state, k2)
    k3 = deriv_func(k3_state, params, coupling_params, spec)
    k4_state = jax.tree_util.tree_map(lambda y, dy: y + dy * dt, state, k3)
    k4 = deriv_func(k4_state, params, coupling_params, spec)

    new_state = jax.tree_util.tree_map(
        lambda y, dy1, dy2, dy3, dy4: y + (dt / 6.0) * (dy1 + 2.0*dy2 + 2.0*dy3 + dy4),
        state, k1, k2, k3, k4
    )
    return new_state

print("SUCCESS: V6 Physics Engine functions defined.")


# --- CELL 6: V6 CERTIFIED EXECUTION FUNCTION ---
# (Source: Technical Implementation Report)

# --- FIX V6.4: REMOVED @partial(jit, ...) decorator. ---
# This function is passed to lax.scan, which will JIT it.
# Compiling it here causes the non-hashable Pytree error.
def jnp_sncgl_conformal_step(
    carry_state: S_NCGL_State,
    t: float,
    # These are the static args that functools.partial will "bake in"
    deriv_func: Callable,
    params: S_NCGL_Params,
    coupling_params: S_Coupling_Params,
    spec: SpecOps,
    # Imported analysis functions
    jnp_construct_conformal_metric: Callable,
    compute_directional_spectrum: Callable,
    compute_log_prime_sse: Callable,
    jnp_calculate_entropy: Callable,
    jnp_calculate_quantule_census: Callable
) -> (S_NCGL_State, dict):
    """Master step function (to be JIT-compiled by lax.scan)."""
    state = carry_state
    DT = params.DT

    # 1. Evolve state with RK4
    new_state = rk4_step(state, DT, deriv_func, params, coupling_params, spec)

    # 2. Compute Metrics & Geometry (for logging)
    new_rho = jnp.abs(new_state.psi)**2

    # (a) Spectral SSE
    k_bins, power_spectrum = compute_directional_spectrum(new_state.psi, params, spec)
    ln_p_sse = compute_log_prime_sse(k_bins, power_spectrum, spec)

    # (b) Aletheia Metrics
    informational_entropy = jnp_calculate_entropy(new_rho)
    quantule_census = jnp_calculate_quantule_census(new_rho)

    # (c) Geometry Metric
    Omega_final_for_log = jnp_construct_conformal_metric(
        new_rho, coupling_params.OMEGA_PARAM_A
    )
    omega_sq_final_for_log = jnp.square(Omega_final_for_log)

    metrics = {
        "timestamp": t * DT,
        "ln_p_sse": ln_p_sse,
        "informational_entropy": informational_entropy,
        "quantule_census": quantule_census,
        "omega_sq_history": omega_sq_final_for_log # Match logger key
    }
    return new_state, metrics

def run_simulation_with_io(
    fmia_params: S_NCGL_Params,
    coupling_params: S_Coupling_Params,
    initial_state: S_NCGL_State,
    spec_ops: SpecOps,
    output_filename="simulation_output.hdf5",
    log_every_n=10
) -> Tuple:
    """
    Orchestrates the S-NCGL simulation, handling JIT compilation
    via functools.partial and managing I/O with the HDF5Logger.
    """
    print("--- Starting Orchestration (S-NCGL V6) ---")

    # 1. Setup simulation parameters
    total_steps = int(fmia_params.T_TOTAL / fmia_params.DT)
    log_steps = total_steps // log_every_n
    if log_steps == 0:
        log_steps = 1

    initial_carry = initial_state
    print(f"Total Steps: {total_steps}, Logging every {log_every_n} steps, Log Steps: {log_steps}")

    # 2. Create the partial function (THE CERTIFIED JIT FIX)
    step_fn_partial = functools.partial(
        jnp_sncgl_conformal_step,
        deriv_func=jnp_get_derivatives,
        params=fmia_params,
        coupling_params=coupling_params,
        spec=spec_ops,
        jnp_construct_conformal_metric=jnp_construct_conformal_metric,
        compute_directional_spectrum=compute_directional_spectrum,
        compute_log_prime_sse=compute_log_prime_sse,
        jnp_calculate_entropy=jnp_calculate_entropy,
        jnp_calculate_quantule_census=jnp_calculate_quantule_census
    )

    # 3. JIT-compile the partial function
    # --- FIX V6.4: REMOVED this line. We do not jit the partial function. ---
    # jit_step_fn = jax.jit(step_fn_partial) # <--- THIS WAS THE BUG

    # Define the "chunk" scanner
    def scan_chunk(carry, _):
        # Run 'log_every_n' steps at once
        # --- FIX V6.4: Pass the *un-compiled* partial function to lax.scan ---
        return lax.scan(step_fn_partial, carry, jnp.arange(log_every_n))

    # JIT-compile the chunk scanner itself
    jit_scan_chunk = jax.jit(scan_chunk)

    # 4. Initialize the Logger
    metrics_to_log = ["timestamp", "ln_p_sse", "informational_entropy", "quantule_census"]
    logger = HDF5Logger(output_filename, log_steps, fmia_params.N_GRID, metrics_to_log)
    print(f"HDF5Logger initialized. Output file: {output_filename}")

    # 5. Run the Main Simulation Loop
    print("--- Starting Simulation Loop (S-NCGL + Geometric Feedback) ---")
    start_time = time.time()
    current_carry = initial_carry

    for i in tqdm(range(log_steps), desc="V6 Sim Progress"):
        try:
            # Call the JIT-compiled chunk scanner
            final_carry_state, metrics_chunk = jit_scan_chunk(current_carry, None)

            # Get the metrics from the *last* step of the chunk
            last_metrics_in_chunk = {
                key: metrics_chunk[key][-1]
                for key in (metrics_to_log + ['omega_sq_history'])
            }

            # Log this data
            logger.log_timestep(last_metrics_in_chunk)
            current_carry = final_carry_state
        except Exception as e:
            print(f"\nERROR during simulation step {i}: {e}")
            logger.close() # Attempt to close logger
            raise # Re-raise the exception

    end_time = time.time()
    print(f"--- Simulation Loop Complete---")
    print(f"Total execution time: {end_time - start_time:.2f} seconds")

    # 6. Save final state and close logger
    logger.save_final_state(current_carry.psi)
    logger.close()

    print(f"Final state (psi hash): {hash(current_carry.psi.tobytes())}")

    # Return success flag
    return current_carry, output_filename, True


# --- CELL 7: (REPLACEMENT) CERTIFIED V6 "WORKER" LOGIC (Corrected) ---
#
# This block replaces the original "Test Harness".
# It contains the complete logic for the "Worker" script,
# allowing this notebook to be called directly by the
# adaptive_hunt_orchestrator.py script.
#
# --- Helper Functions (for the Worker) ---

def generate_param_hash(params: Dict[str, Any]) -> str:
    """Creates a unique SHA256 hash from a parameter dictionary."""
    # We must sort the dictionary to ensure consistent hashes
    sorted_params_str = json.dumps(params, sort_keys=True).encode('utf-8')
    # hashlib was imported in Cell 1
    hash_str = hashlib.sha256(sorted_params_str).hexdigest()
    return hash_str[:12]

def write_to_ledger(ledger_file: str, run_data: Dict[str, Any]):
    """Appends a single run's data to the CSV ledger."""
    file_exists = os.path.isfile(ledger_file)
    all_headers = sorted(list(run_data.keys()))

    preferred_order = [
        'param_hash', 'final_sse', 'jax_run_seed', 'generation',
        'alpha', 'sigma_k', 'nu', 'OMEGA_PARAM_A', 'KAPPA',
        'gamma', 'beta', 'N_GRID', 'T_TOTAL'
    ]

    final_headers = [h for h in preferred_order if h in all_headers] + \
                     [h for h in all_headers if h not in preferred_order]

    # Clean run_data: ensure all values are serializable
    cleaned_run_data = {}
    for k, v in run_data.items():
        if isinstance(v, (float, np.floating)) and (np.isnan(v) or np.isinf(v)):
            cleaned_run_data[k] = -999.0 # Use a placeholder
        else:
            cleaned_run_data[k] = v

    try:
        # csv was imported in Cell 1
        with open(ledger_file, 'a', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=final_headers, extrasaction='ignore')
            if not file_exists:
                writer.writeheader()
            writer.writerow(cleaned_run_data)
    except Exception as e:
        print(f"  > [WORKER] Error writing to ledger: {e}")

def load_todo_list(todo_file: str) -> List[Dict[str, Any]]:
    """Loads the list of jobs from the Hunter."""
    try:
        with open(todo_file, 'r') as f:
            # FIX V6.2: Added json.load(f) to parse the file content.
            jobs = json.load(f)

        os.remove(todo_file)
        print(f"  > [WORKER] Loaded and removed '{todo_file}'.")
        return jobs
    except FileNotFoundError:
        return []
    except json.JSONDecodeError:
        print(f"  > [WORKER] ERROR: '{todo_file}' is corrupted or empty. Deleting.")
        os.remove(todo_file)
        return []

def generate_bootstrap_jobs(
    rng: np.random.Generator, num_jobs: int
) -> List[Dict[str, Any]]:
    """Creates the 'Generation 0' for the "Blind 5D Exploration" hunt."""
    print(f"  > [WORKER] Generating {num_jobs} (5D BLIND) bootstrap jobs (Gen 0)...")
    jobs = []

    # These are the wide, 5D ranges for our "Blind" hunt
    PARAM_RANGES = {
        'alpha':         ('uniform', 0.01, 1.0),
        'sigma_k':       ('uniform', 0.1, 10.0),
        'nu':            ('uniform', 0.1, 5.0),
        'OMEGA_PARAM_A': ('uniform', 0.1, 2.5),
        'KAPPA':         ('uniform', 0.001, 5.0)
    }

    print(f"  > [WORKER] Generating {num_jobs} random 'immigrants'...")
    for _ in range(num_jobs):
        job = {}
        for key, (dist, p_min, p_max) in PARAM_RANGES.items():
            if dist == 'uniform':
                job[key] = rng.uniform(low=p_min, high=p_max)
        job['generation'] = 0
        jobs.append(job)
    return jobs

def run_worker_main(hunt_id, todo_file):
    """This is the main "Worker" function that the orchestrator calls."""
    print(f"--- [WORKER] ENGAGED for {hunt_id} (V6 Engine) ---")

    MASTER_SEED = 42
    BOOTSTRAP_JOBS = 100 # Must match GENERATION_SIZE in hunter

    # Static physics params (non-evolvable)
    STATIC_PHYSICS_PARAMS = {
        "gamma": 0.2,
        "beta": 1.0,
        "N_GRID": 128,
        "T_TOTAL": 2.0,
        "DT": 1e-3
    }

    # Static simulation setup params
    L_DOMAIN = 20.0
    K_MAX_PLOT = 2.0
    K_BIN_WIDTH = 0.01
    NUM_RAYS = 32
    LOG_EVERY_N_STEPS = 10 # HDF5 logging frequency

    # Setup directories and RNG
    MASTER_OUTPUT_DIR = os.path.join("sweep_runs", hunt_id)
    os.makedirs(MASTER_OUTPUT_DIR, exist_ok=True)
    LEDGER_FILE = os.path.join(MASTER_OUTPUT_DIR, f"ledger_{hunt_id}.csv")
    master_rng = np.random.default_rng(MASTER_SEED)

    # --- Load or Generate Job List ---
    params_to_run = load_todo_list(todo_file)
    if not params_to_run:
        print(f"  > [WORKER] No '{todo_file}' found. Bootstrapping (5D Blind)...")
        params_to_run = generate_bootstrap_jobs(master_rng, BOOTSTRAP_JOBS)

    total_jobs = len(params_to_run)
    print(f"  > [WORKER] Found {total_jobs} jobs to run.")

    sweep_start_time = time.time()

    # --- Loop over all jobs from the Hunter ---
    for i, variable_params in enumerate(params_to_run):
        run_start_time = time.time()
        print(f"\n  --- [WORKER] Starting Job {i+1} / {total_jobs} ---")

        # FIX V6.2: Check for corrupted data
        if not isinstance(variable_params, dict):
            print(f"!!! [WORKER] ERROR: Invalid job format. Expected dict, got {type(variable_params)}. Skipping job.")
            print(f"    Bad data: {variable_params}")
            continue

        # 1. Combine static and variable params
        current_run_params = variable_params.copy()
        current_run_params.update(STATIC_PHYSICS_PARAMS)

        # 2. Add generation, seed, and hash
        if 'generation' not in current_run_params:
            current_run_params['generation'] = 'unknown'

        jax_run_seed = int(master_rng.integers(low=0, high=2**31 - 1))
        current_run_params['jax_run_seed'] = jax_run_seed
        param_hash = generate_param_hash(current_run_params)
        current_run_params['param_hash'] = param_hash
        print(f"    Run Hash: {param_hash} | JAX Seed: {jax_run_seed}")

        # 3. Assemble the V6 JAX Pytrees (Structs)
        try:
            fmia_params = S_NCGL_Params(
                N_GRID=int(current_run_params["N_GRID"]),
                T_TOTAL=float(current_run_params["T_TOTAL"]),
                DT=float(current_run_params["DT"]),
                alpha=float(current_run_params["alpha"]),
                beta=float(current_run_params["beta"]),
                gamma=float(current_run_params["gamma"]),
                KAPPA=float(current_run_params["KAPPA"]),
                nu=float(current_run_params["nu"]),
                sigma_k=float(current_run_params["sigma_k"]),
                l_domain=L_DOMAIN,
                num_rays=NUM_RAYS,
                k_bin_width=K_BIN_WIDTH,
                k_max_plot=K_MAX_PLOT
            )

            # FIX V6.3: Create a hashable S_Coupling_Params NamedTuple
            coupling_params = S_Coupling_Params(
                OMEGA_PARAM_A=float(current_run_params["OMEGA_PARAM_A"])
            )

            key = jax.random.PRNGKey(jax_run_seed)
            N_GRID = fmia_params.N_GRID

            # This call is safe due to the V6.2 fix in Cell 4
            kx, ky, k_sq, k_mag, k_max_sim, dealias_mask = kgrid_2pi(N_GRID, L_DOMAIN)

            gaussian_kernel_k = make_gaussian_kernel_k(k_sq, fmia_params.sigma_k)
            k_bins = jnp.arange(0, K_MAX_PLOT, K_BIN_WIDTH)
            primes = jnp.array([2, 3, 5, 7, 11, 13, 17, 19])
            prime_targets_k = jnp.log(primes)

            xx, yy = jnp.meshgrid(
                jnp.linspace(-0.5, 0.5, N_GRID) * L_DOMAIN,
                jnp.linspace(-0.5, 0.5, N_GRID) * L_DOMAIN
            )
            k_values_1d = 2 * jnp.pi * jnp.fft.fftfreq(N_GRID, d=L_DOMAIN / N_GRID)
            sort_indices_1d = jnp.argsort(k_values_1d)

            spec_ops = SpecOps(
                kx=kx.astype(jnp.float32),
                ky=ky.astype(jnp.float32),
                k_sq=k_sq.astype(jnp.float32),
                gaussian_kernel_k=gaussian_kernel_k.astype(jnp.float32),
                dealias_mask=dealias_mask.astype(jnp.float32),
                k_bins=k_bins.astype(jnp.float32),
                prime_targets_k=prime_targets_k.astype(jnp.float32),
                ray_angles=jnp.linspace(0, jnp.pi, NUM_RAYS),
                k_max=k_max_sim.astype(jnp.float32),
                xx=xx.astype(jnp.float32),
                yy=yy.astype(jnp.float32),
                k_values_1d=k_values_1d.astype(jnp.float32),
                sort_indices_1d=sort_indices_1d.astype(jnp.int32)
            )

            psi_initial = (
                jax.random.uniform(key, (N_GRID, N_GRID), dtype=jnp.float32) * 0.1 +
                1j * jax.random.uniform(key, (N_GRID, N_GRID), dtype=jnp.float32) * 0.1
            )
            initial_state = S_NCGL_State(psi=psi_initial.astype(jnp.complex64))

            output_filename = os.path.join(MASTER_OUTPUT_DIR, f"run_{param_hash}.hdf5")

        except Exception as e:
            print(f"!!! [WORKER] JOB {param_hash} FAILED during parameter assembly: {e} !!!")
            traceback.print_exc()
            final_sse = 99998.0 # Assembly error code
            current_run_params['final_sse'] = final_sse
            write_to_ledger(LEDGER_FILE, current_run_params)
            continue # Skip to next job

        # 4. Run the V6 Simulation
        sim_success = False
        try:
            # This function is defined in Cell 6
            final_carry_state, output_file, sim_success = run_simulation_with_io(
                fmia_params,
                coupling_params, # This is now a hashable S_Coupling_Params
                initial_state,
                spec_ops,
                output_filename=output_filename,
                log_every_n=LOG_EVERY_N_STEPS
            )

            # 5. Get the Final SSE
            if sim_success:
                with h5py.File(output_file, 'r') as f:
                    final_sse = float(f['ln_p_sse'][-1])
            else:
                final_sse = 99999.0 # Sim failed internally

        except Exception as e:
            print(f"!!! [WORKER] JOB {param_hash} FAILED during simulation: {e} !!!")
            traceback.print_exc()
            final_sse = 99999.0 # Sim crash error code

        run_end_time = time.time()

        # 6. Log results to master ledger
        current_run_params['final_sse'] = final_sse
        print(f"  --- [WORKER] Job {i+1} Complete ({run_end_time - run_start_time:.2f}s) ---")
        print(f"    Final SSE: {final_sse:.12f}")
        write_to_ledger(LEDGER_FILE, current_run_params)

    # --- Loop Finished ---
    sweep_end_time = time.time()
    print(f"\n--- [WORKER] FINISHED {hunt_id} ---")
    print(f"Total time for {total_jobs} jobs: {(sweep_end_time - sweep_start_time) / 60.0:.2f} minutes")


# --- THIS IS THE NEW "MAIN" BLOCK ---
if __name__ == "__main__":

    # This block allows the notebook to be called
    # as a script by adaptive_hunt_orchestrator.py

    # --- Check for dependencies (for Colab) ---
    try:
        # FIX V6.2: Corrected the tqdm TypeError
        import jax, pandas, h5py
        print("All dependencies satisfied.")
    except ImportError:
        print("Installing dependencies (jax, pandas, h5py, tqdm, matplotlib)...")
        import subprocess
        subprocess.run(["pip", "install", "--quiet", "jax", "jaxlib", "pandas", "h5py", "tqdm", "matplotlib"], check=True)
        print("Dependency installation complete. Please RESTART the runtime if imports fail.")

    # --- Main Logic ---
    if len(sys.argv) < 3: # Check if not enough args
        print("\n" + "="*50)
        print("--- [WORKER] Running in TEST MODE ---")
        print("No CLI args detected. This will run one test simulation.")
        print("="*50)

        # This is the "Test Harness" logic
        HUNT_ID = "SNCGL_ADAPTIVE_HUNT_TEST"
        TODO_FILE = "ASTE_generation_todo_TEST.json"

        # Create a dummy todo file for the test
        test_params = {
            "alpha": 0.1, "KAPPA": 1.0, "nu": 1.0,
            "sigma_k": 2.5, "OMEGA_PARAM_A": 0.5,
            "generation": -1 # Test generation
        }
        with open(TODO_FILE, 'w') as f:
            json.dump([test_params], f)

        # Run the worker main function
        run_worker_main(HUNT_ID, TODO_FILE)

    else:
        # --- This is the PRODUCTION logic ---
        # This block runs when called by the orchestrator
        print(f"--- [WORKER] Production mode activated by orchestrator ---")
        HUNT_ID = sys.argv[1]
        TODO_FILE = sys.argv[2]
        run_worker_main(hunt_id=HUNT_ID, todo_file=TODO_FILE)

print("worker_v6.py successfully written.")

Writing worker_v6.py


In [ ]:
%%writefile aste_hunter.py
import pandas as pd
import numpy as np
import os
import json
import traceback
import sys
from typing import Dict, Any, List, Tuple

print("--- [HUNTER] ENGAGED (v6: 5D BLIND Hunt) ---")

# --- 1. Configuration: Genetic Algorithm Settings ---
GENERATION_SIZE = 100
ELITE_PERCENTILE = 0.1
MUTATION_STRENGTH = 0.15  # Aggressive mutation
MUTATION_CHANCE = 0.7
STAGNATION_GENERATIONS = 3 # Check if best SSE is stuck for 3 gens
IMMIGRANT_RATIO = 0.5     # If stuck, 50% of next gen is random

# --- 5D FULL-SPACE (WIDE) PARAMETER RANGES ---
# These MUST match the "Worker" notebook's bootstrap function
PARAMS_TO_EVOLVE = ['alpha', 'sigma_k', 'nu', 'OMEGA_PARAM_A', 'KAPPA']
PARAM_RANGES = {
    'alpha':         ('uniform', 0.01, 1.0),
    'sigma_k':       ('uniform', 0.1, 10.0),
    'nu':            ('uniform', 0.1, 5.0),
    'OMEGA_PARAM_A': ('uniform', 0.1, 2.5),
    'KAPPA':         ('uniform', 0.001, 5.0)
}

# --- 2. Helper Functions ---
def clamp_param(value: float, p_min: float, p_max: float) -> float:
    """Clamps a value within the valid min/max range."""
    return max(p_min, min(value, p_max))

def generate_random_immigrants(rng: np.random.Generator, num_immigrants: int, generation: int) -> List[Dict[str, Any]]:
    """Generates new random candidates (immigrants) for exploration."""
    immigrants = []
    for _ in range(num_immigrants):
        job = {}
        for key, (dist, p_min, p_max) in PARAM_RANGES.items():
            if dist == 'uniform':
                 job[key] = rng.uniform(low=p_min, high=p_max)
        job['generation'] = generation
        immigrants.append(job)
    return immigrants

def check_stagnation(ledger_df: pd.DataFrame, current_gen: int, threshold: int) -> bool:
    """Checks if the best SSE has improved in the last 'threshold' generations."""
    if current_gen < threshold:
        return False
    try:
        # Get best SSE for the last 'threshold' generations
        gen_indices = range(current_gen - threshold + 1, current_gen + 1)
        best_sses = []
        for gen in gen_indices:
            gen_runs = ledger_df[ledger_df['generation'] == gen]
            if gen_runs.empty: continue # Skip gen if no data
            best_sses.append(gen_runs['final_sse'].min())

        if len(best_sses) < threshold:
            return False # Not enough data to be sure

        # Check if all recent best SSEs are effectively the same
        first_sse = best_sses[0]
        if all(np.isclose(sse, first_sse, atol=1e-6) for sse in best_sses):
            print(f"  > [HUNTER] STAGNATION DETECTED. Best SSE stuck at {first_sse:.6f}.")
            return True
    except Exception as e:
         print(f"  > [HUNTER] Warning: Could not check stagnation. {e}")

    return False

# --- 3. Main Hunter Logic ---
def main(hunt_id, todo_file):
    """
    Analyzes the ledger, detects stagnation, and breeds a new generation
    of parameters, saving them to the 'todo_file'.
    """
    print(f"  > [HUNTER] Analyzing {hunt_id}...")
    MASTER_OUTPUT_DIR = os.path.join("sweep_runs", hunt_id)
    LEDGER_FILE = os.path.join(MASTER_OUTPUT_DIR, f"ledger_{hunt_id}.csv")
    rng = np.random.default_rng()

    try:
        if os.path.exists(todo_file):
             print(f"  > [HUNTER] Error: '{todo_file}' already exists.")
            print("    The 'Worker' has not run the last generation.")
            return

        if not os.path.exists(LEDGER_FILE):
            print(f"  > [HUNTER] Ledger not found: {LEDGER_FILE}")
            print("    Cannot run. Please bootstrap the 'Worker' first.")
            return

        ledger_df = pd.read_csv(LEDGER_FILE)
        # Filter out failed runs
        valid_runs_df = ledger_df[ledger_df['final_sse'] < 90000].copy()

        if valid_runs_df.empty:
            print("  > [HUNTER] Ledger contains no valid runs. Cannot breed.")
            return

        # Find the best runs ("elite") from the *entire* history
        n_elite = max(2, int(len(valid_runs_df) * ELITE_PERCENTILE))
        elite_df = valid_runs_df.sort_values(by='final_sse').head(n_elite)

        current_gen = int(ledger_df['generation'].max())
        next_gen = current_gen + 1

        print(f"  > [HUNTER] Loaded {len(ledger_df)} total runs.")
        print(f"  > [HUNTER] Current Gen: {current_gen}, Best SSE: {elite_df['final_sse'].min():.8f}")

        # --- Diversity Injection Logic ---
        is_stagnant = check_stagnation(valid_runs_df, current_gen, STAGNATION_GENERATIONS)
        num_immigrants = 0

        if is_stagnant:
            print(f"  > [HUNTER] Injecting diversity: {IMMIGRANT_RATIO*100}% immigrants.")
            num_immigrants = int(GENERATION_SIZE * IMMIGRANT_RATIO)

        num_children = GENERATION_SIZE - num_immigrants
        new_generation_params = []

        # 1. Breed Children (Exploitation)
        print(f"  > [HUNTER] Breeding {num_children} 'children'...")
        for i in range(num_children):
            parent1_series = elite_df.sample(1).iloc[0]
            parent2_series = elite_df.sample(1).iloc[0]
            child = {}
            for key in PARAMS_TO_EVOLVE:
                # Get params, falling back to a random value if key is missing
                p1 = parent1_series.get(key, rng.uniform(PARAM_RANGES[key][1], PARAM_RANGES[key][2]))
                p2 = parent2_series.get(key, rng.uniform(PARAM_RANGES[key][1], PARAM_RANGES[key][2]))
                child[key] = (p1 + p2) / 2.0

                if rng.random() < MUTATION_CHANCE:
                    mutation_val = rng.normal(0, MUTATION_STRENGTH * child[key])
                    child[key] += mutation_val
                    # Clamp the value
                    p_min, p_max = PARAM_RANGES[key][1], PARAM_RANGES[key][2]
                    child[key] = clamp_param(child[key], p_min, p_max)

            child['generation'] = next_gen
            new_generation_params.append(child)

        # 2. Generate Immigrants (Exploration)
        if num_immigrants > 0:
            print(f"  > [HUNTER] Generating {num_immigrants} 'immigrants'...")
            immigrants = generate_random_immigrants(rng, num_immigrants, next_gen)
            new_generation_params.extend(immigrants)

        # 3. Save the new "to-do" list
        with open(todo_file, 'w') as f:
            json.dump(new_generation_params, f, indent=2)

        print(f"\n--- [HUNTER] FINISHED ---")
        print(f"  > Successfully created '{todo_file}' for Gen {next_gen} ({num_children} children, {num_immigrants} immigrants).")

    except Exception as e:
        print(f"\n--- [HUNTER] FAILED ---")
        print(f"An error occurred: {e}")
        traceback.print_exc()

if __name__ == "__main__":
    if len(sys.argv) != 3:
        print("Usage: python aste_hunter.py <HUNT_ID> <TODO_FILE_NAME>")
        sys.exit(1)

    # This allows the orchestrator to call this script
    main(hunt_id=sys.argv[1], todo_file=sys.argv[2])

print("aste_hunter.py successfully written.")

Writing aste_hunter.py


In [ ]:
%%writefile adaptive_hunt_orchestrator.py
import os
import subprocess
import pandas as pd
import time
import sys
import shlex # For handling filenames with spaces

print("--- [ORCHESTRATOR] ENGAGED (v10: 5D BLIND Hunt) ---")

# --- 1. Global Configuration Parameters ---
GOAL_SSE = 0.1  # Our first target is to beat the 1.2 local minimum
CONSECUTIVE_GOAL_GENS = 3
NUM_HUNTS = 3
MASTER_OUTPUT_DIR = "sweep_runs"
TODO_FILE = "ASTE_generation_todo.json"
HUNT_ID_OFFSET = 30  # Start from HUNT_030 (clean slate)

# *** CRITICAL: This now points to our new v6.2 .py file ***
WORKER_NOTEBOOK_FILE = "worker_v6.py"

# --- 2. Helper Functions ---
def run_command(command_parts):
    """Executes a shell command and prints its output in real-time."""
    # Use shlex.quote to handle filenames with spaces/special chars
    print(f"\nExecuting: {' '.join(shlex.quote(part) for part in command_parts)}\n")

    process = subprocess.Popen(command_parts,
                             stdout=subprocess.PIPE,
                             stderr=subprocess.STDOUT,
                             text=True,
                             encoding='utf-8')
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print(output.strip())
    rc = process.poll()
    if rc != 0:
        print(f"\n[ORCHESTRATOR] Command failed with exit code {rc}")
    return rc

def get_best_sse(hunt_id):
    """Reads the ledger and returns the best SSE for a given hunt_id."""
    ledger_file = os.path.join(MASTER_OUTPUT_DIR, hunt_id, f"ledger_{hunt_id}.csv")
    if not os.path.exists(ledger_file):
        return float('inf')
    try:
        ledger_df = pd.read_csv(ledger_file)
        valid_runs = ledger_df[ledger_df['final_sse'] < 90000]
        if valid_runs.empty:
            return float('inf')
        return valid_runs['final_sse'].min()
    except Exception:
        return float('inf')

# --- 3. Main Orchestrator Logic ---
def main():
    # Debugging: List files and current directory
    print(f"Orchestrator's CWD: {os.getcwd()}")
    print(f"Checking for worker file: {WORKER_NOTEBOOK_FILE}")
    print("Files in orchestrator's CWD:")
    # Using subprocess.run to ensure 'ls -l' output is captured and printed
    subprocess.run(["ls", "-l"], stdout=sys.stdout, stderr=sys.stderr)
    print("-" * 30)

    # Verify the Worker notebook exists before starting
    if not os.path.exists(WORKER_NOTEBOOK_FILE):
        print(f"--- [ORCHESTRATOR] CRITICAL ERROR ---")
        print(f"Worker notebook not found: '{WORKER_NOTEBOOK_FILE}'")
        print("Please ensure the 'WORKER_NOTEBOOK_FILE' variable is set correctly.")
        return

    print(f"--- Orchestrator: Targeting SSE < {GOAL_SSE} for {CONSECUTIVE_GOAL_GENS} generations ---")

    for i in range(NUM_HUNTS):
        hunt_index = i + HUNT_ID_OFFSET
        HUNT_ID = f"SNCGL_ADAPTIVE_HUNT_{hunt_index:003d}"

        print(f"\n{'-'*80}")
        print(f"--- STARTING ADAPTIVE HUNT: {HUNT_ID} (5D Blind Exploration) ---")
        print(f"{'-'*80}")

        consecutive_goal_met_gens = 0
        generation_counter = 0

        while True:
            print(f"\n--- Hunt {HUNT_ID}, Generation {generation_counter} ---")

            # --- Step 1: Run Worker ---
            print(f"Calling Worker: {WORKER_NOTEBOOK_FILE}...")
            # We call the notebook as a python script.
            # The 'if __name__ == "__main__":' block inside it handles this.
            worker_command = [
                "python3", # Use python3
                WORKER_NOTEBOOK_FILE,
                HUNT_ID,
                TODO_FILE
            ]
            rc = run_command(worker_command)
            if rc != 0:
                print(f"[ORCHESTRATOR] Worker failed for {HUNT_ID}. Stopping hunt.")
                break

            # --- Step 2: Run Hunter ---
            print(f"Calling Hunter: aste_hunter.py...")
            hunter_command = [
                "python3",
                "aste_hunter.py",
                HUNT_ID,
                TODO_FILE
            ]
            rc = run_command(hunter_command)
            if rc != 0:
                print(f"[ORCHESTRATOR] Hunter failed for {HUNT_ID}. Stopping hunt.")
                break

            # --- Step 3: Monitor best SSE ---
            current_best_sse = get_best_sse(HUNT_ID)
            print(f"Hunt {HUNT_ID}, Generation {generation_counter}: Current Best SSE = {current_best_sse:.12f}")

            if current_best_sse <= GOAL_SSE:
                consecutive_goal_met_gens += 1
                print(f"GOAL SSE MET! Consecutive generations: {consecutive_goal_met_gens}/{CONSECUTIVE_GOAL_GENS}")
            else:
                consecutive_goal_met_gens = 0
                print("GOAL SSE NOT MET. Resetting consecutive counter.")

            if consecutive_goal_met_gens >= CONSECUTIVE_GOAL_GENS:
                print(f"\n--- Hunt {HUNT_ID} COMPLETED ---")
                print(f"GOAL SSE ({GOAL_SSE}) achieved for {CONSECUTIVE_GOAL_GENS} consecutive generations!")
                break

            if generation_counter > 50: # Safety break
                print(f"\n--- Hunt {HUNT_ID} STOPPED ---")
                print(f"Did not meet goal after 50 generations. Moving to next hunt.")
                break

            generation_counter += 1
            time.sleep(1) # Small delay

        # Clean up any leftover todo file before next hunt
        if os.path.exists(TODO_FILE):
            os.remove(TODO_FILE)
            print(f"Cleaned up residual '{TODO_FILE}'.")

    print(f"\n{'-'*80}")
    print("--- ORCHESTRATOR FINISHED ALL HUNTS ---")

if __name__ == "__main__":
    main()

print("adaptive_hunt_orchestrator.py successfully written.")

Writing adaptive_hunt_orchestrator.py


In [ ]:
!python3 adaptive_hunt_orchestrator.py

--- [ORCHESTRATOR] ENGAGED (v10: 5D BLIND Hunt) ---
Orchestrator's CWD: /content
Checking for worker file: worker_v6.py
Files in orchestrator's CWD:
total 56
-rw-r--r-- 1 root root  5743 Nov  5 14:49 adaptive_hunt_orchestrator.py
-rw-r--r-- 1 root root  6944 Nov  5 14:49 aste_hunter.py
drwx------ 5 root root  4096 Nov  5 14:49 drive
drwxr-xr-x 1 root root  4096 Nov  3 14:39 sample_data
-rw-r--r-- 1 root root 30074 Nov  5 14:49 worker_v6.py
------------------------------
--- Orchestrator: Targeting SSE < 0.1 for 3 generations ---

--------------------------------------------------------------------------------
--- STARTING ADAPTIVE HUNT: SNCGL_ADAPTIVE_HUNT_030 (5D Blind Exploration) ---
--------------------------------------------------------------------------------

--- Hunt SNCGL_ADAPTIVE_HUNT_030, Generation 0 ---
Calling Worker: worker_v6.py...

Executing: python3 worker_v6.py SNCGL_ADAPTIVE_HUNT_030 ASTE_generation_todo.json

JAX backend: cpu
SUCCESS: V6 Analysis & Geometry functi

# Task
Execute the adaptive hunt orchestrated by `adaptive_hunt_orchestrator.py`.

## Verify Script Files

### Subtask:
Confirm the presence and readability of the 'worker_v6.py', 'aste_hunter.py', and 'adaptive_hunt_orchestrator.py' files in the current directory. This step ensures all necessary components for the adaptive hunt are accessible.


**Reasoning**:
To confirm the presence and readability of the specified script files, I will first list all files in the current directory. This will visually verify their existence and also display their permissions and sizes, providing an initial check of their integrity.



In [ ]:
import os

# List files in the current directory to confirm presence
print("Listing files in current directory:")
!ls -l

# Define the files to verify
files_to_verify = [
    'worker_v6.py',
    'aste_hunter.py',
    'adaptive_hunt_orchestrator.py'
]

print("\nVerifying readability of script files...")
read_success = True
for filename in files_to_verify:
    if os.path.exists(filename):
        try:
            with open(filename, 'r') as f:
                # Read the first 5 lines to check readability
                head = [next(f) for _ in range(5)]
                print(f"  - '{filename}' exists and is readable (first 5 lines:\n    {''.join(head).strip().replace('\n', '\n    ')})\n")
        except Exception as e:
            print(f"  - ERROR: Could not read '{filename}': {e}")
            read_success = False
    else:
        print(f"  - ERROR: '{filename}' does not exist.")
        read_success = False

if read_success:
    print("All specified script files exist and are readable.")
else:
    print("WARNING: Some script files were not found or could not be read.")


Listing files in current directory:
total 64
-rw-r--r-- 1 root root  5743 Nov  5 14:49  adaptive_hunt_orchestrator.py
-rw-r--r-- 1 root root  6944 Nov  5 14:49  aste_hunter.py
drwx------ 5 root root  4096 Nov  5 14:49  drive
drwxr-xr-x 1 root root  4096 Nov  3 14:39  sample_data
drwxr-xr-x 3 root root  4096 Nov  5 14:49  sweep_runs
-rw-r--r-- 1 root root  3990 Nov  5 14:53 "This is a fantastic development. I'.txt"
-rw-r--r-- 1 root root 30074 Nov  5 14:49  worker_v6.py

Verifying readability of script files...
  - 'worker_v6.py' exists and is readable (first 5 lines:
    #
    # worker_v6.py (Certified v6.4 - Final JAX Blocker Fix)
    #
    # This script is the "Certified V6 Worker" for the ASTE.
    # It contains all code (Engine, Scorer, Bridge, HPC Pattern, and Worker Logic))

  - 'aste_hunter.py' exists and is readable (first 5 lines:
    import pandas as pd
    import numpy as np
    import os
    import json
    import traceback)

  - 'adaptive_hunt_orchestrator.py' exists and i

## Review Orchestrator Configuration

### Subtask:
Inspect the 'adaptive_hunt_orchestrator.py' script for key configuration parameters such as 'GOAL_SSE', 'CONSECUTIVE_GOAL_GENS', 'NUM_HUNTS', and 'HUNT_ID_OFFSET'. This allows for an initial check of the hunt's objectives and scope.


**Reasoning**:
The next step is to read the content of 'adaptive_hunt_orchestrator.py' and extract the values for the specified configuration parameters.



In [ ]:
import re

orchestrator_config = {}
config_vars = [
    'GOAL_SSE',
    'CONSECUTIVE_GOAL_GENS',
    'NUM_HUNTS',
    'HUNT_ID_OFFSET'
]

with open('adaptive_hunt_orchestrator.py', 'r') as f:
    content = f.read()

for var in config_vars:
    match = re.search(f"^{var}\s*=\s*(.+)", content, re.MULTILINE)
    if match:
        value = match.group(1).strip()
        try:
            # Try to convert to appropriate type if possible
            if '.' in value:
                orchestrator_config[var] = float(value)
            else:
                orchestrator_config[var] = int(value)
        except ValueError:
            orchestrator_config[var] = value

print("Orchestrator Configuration Parameters:")
for key, val in orchestrator_config.items():
    print(f"  - {key}: {val}")

Orchestrator Configuration Parameters:
  - GOAL_SSE: 0.1  # Our first target is to beat the 1.2 local minimum
  - CONSECUTIVE_GOAL_GENS: 3
  - NUM_HUNTS: 3
  - HUNT_ID_OFFSET: 30  # Start from HUNT_030 (clean slate)


<>:15: SyntaxWarning: invalid escape sequence '\s'
<>:15: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-2154172386.py:15: SyntaxWarning: invalid escape sequence '\s'
  match = re.search(f"^{var}\s*=\s*(.+)", content, re.MULTILINE)


**Reasoning**:
The previous code produced a SyntaxWarning for an invalid escape sequence and extracted values with comments. I will fix the regex pattern to use a raw f-string and accurately capture only the numerical values for the configuration parameters, ignoring comments.



In [ ]:
import re

orchestrator_config = {}
config_vars = [
    'GOAL_SSE',
    'CONSECUTIVE_GOAL_GENS',
    'NUM_HUNTS',
    'HUNT_ID_OFFSET'
]

with open('adaptive_hunt_orchestrator.py', 'r') as f:
    content = f.read()

for var in config_vars:
    # Refined regex to capture only the number, ignoring comments and using raw f-string
    match = re.search(rf"^{var}\s*=\s*(\d+(\.\d+)?)\s*(?:#.*)?", content, re.MULTILINE)
    if match:
        value = match.group(1).strip() # Extract the captured numerical part
        try:
            if '.' in value:
                orchestrator_config[var] = float(value)
            else:
                orchestrator_config[var] = int(value)
        except ValueError:
            # Fallback if conversion fails, though with refined regex it should be rare
            orchestrator_config[var] = value

print("Orchestrator Configuration Parameters:")
for key, val in orchestrator_config.items():
    print(f"  - {key}: {val}")

Orchestrator Configuration Parameters:
  - GOAL_SSE: 0.1
  - CONSECUTIVE_GOAL_GENS: 3
  - NUM_HUNTS: 3
  - HUNT_ID_OFFSET: 30


## Execute Adaptive Hunt Orchestrator

### Subtask:
Run the 'adaptive_hunt_orchestrator.py' script. This script will sequentially call the worker and hunter components to perform the adaptive hunt, generate new parameters, and log results. This step is critical for implementing the overall system.


**Reasoning**:
To execute the adaptive hunt orchestrator as requested, I will use a shell command to run the Python script.



In [ ]:
!python3 adaptive_hunt_orchestrator.py

--- [ORCHESTRATOR] ENGAGED (v10: 5D BLIND Hunt) ---
Orchestrator's CWD: /content
Checking for worker file: worker_v6.py
Files in orchestrator's CWD:
total 64
-rw-r--r-- 1 root root  5743 Nov  5 14:49  adaptive_hunt_orchestrator.py
-rw-r--r-- 1 root root  6944 Nov  5 14:49  aste_hunter.py
drwx------ 5 root root  4096 Nov  5 14:49  drive
drwxr-xr-x 1 root root  4096 Nov  3 14:39  sample_data
drwxr-xr-x 3 root root  4096 Nov  5 14:49  sweep_runs
-rw-r--r-- 1 root root  3990 Nov  5 14:53 "This is a fantastic development. I'.txt"
-rw-r--r-- 1 root root 30074 Nov  5 14:49  worker_v6.py
------------------------------
--- Orchestrator: Targeting SSE < 0.1 for 3 generations ---

--------------------------------------------------------------------------------
--- STARTING ADAPTIVE HUNT: SNCGL_ADAPTIVE_HUNT_030 (5D Blind Exploration) ---
--------------------------------------------------------------------------------

--- Hunt SNCGL_ADAPTIVE_HUNT_030, Generation 0 ---
Calling Worker: worker_v6.py

# Task
Run the `adaptive_hunt_orchestrator.py` script to resume the adaptive hunt for optimal parameters.

## Re-execute Adaptive Hunt Orchestrator

### Subtask:
Re-run the `adaptive_hunt_orchestrator.py` script. The script is designed to pick up from where it left off, either continuing a generation or starting a new one based on the existing ledger and 'todo' file status. This will resume the adaptive hunt.


**Reasoning**:
To re-execute the adaptive hunt orchestrator, I will run the Python script using the `!python3` command in a code cell. The orchestrator script is designed to pick up from where it left off, based on existing files.



In [ ]:
!python3 adaptive_hunt_orchestrator.py

Streaming output truncated to the last 5000 lines.
V6 Sim Progress: 100%|██████████| 200/200 [00:14<00:00, 13.36it/s]
--- Simulation Loop Complete---
Total execution time: 14.97 seconds
HDF5Logger closed. Data saved to sweep_runs/SNCGL_ADAPTIVE_HUNT_030/run_123c985efdfd.hdf5
Final state (psi hash): 8249011249994388284
--- [WORKER] Job 72 Complete (14.98s) ---
Final SSE: 1.186931848526

--- [WORKER] Starting Job 73 / 100 ---
Run Hash: bd5d4038ae33 | JAX Seed: 1624628300
--- Starting Orchestration (S-NCGL V6) ---
Total Steps: 2000, Logging every 10 steps, Log Steps: 200
HDF5Logger initialized. Output file: sweep_runs/SNCGL_ADAPTIVE_HUNT_030/run_bd5d4038ae33.hdf5
--- Starting Simulation Loop (S-NCGL + Geometric Feedback) ---

V6 Sim Progress: 100%|██████████| 200/200 [00:14<00:00, 13.42it/s]
--- Simulation Loop Complete---
Total execution time: 14.91 seconds
HDF5Logger closed. Data saved to sweep_runs/SNCGL_ADAPTIVE_HUNT_030/run_bd5d4038ae33.hdf5
Final state (psi hash): 432300241323441213